In [13]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk
import threading
import os

In [14]:
# Load YOLO model
net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
with open('cfg/coco.names', 'r') as f:
    classes = f.read().splitlines()

def detect_objects_yolo(frame):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Handle different return types from NMSBoxes in different OpenCV versions
    if len(indexes) > 0:
        indexes = indexes.flatten() if hasattr(indexes, 'flatten') else indexes.reshape(-1)
        
        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(boxes), 3))
        
        for i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence}", (x, y + 20), font, 1, (255, 255, 255), 2)

    return frame

In [18]:
class ObjectDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PAU Object Detection - YOLO")
        self.root.geometry("800x700")

        # Video file paths
        self.video_paths = {
            "School Of Science and Technology": "videos/SST.mp4",
            "Marian Grotto": "videos/Marian Grotto.mp4",
            "Yemisi Shyllon Museum": "videos/YSMA.mp4"
        }

        self.selected_video = None
        self.cap = None
        self.running = False
        self.paused = False
        self.current_frame = None

        self.create_widgets()

    def create_widgets(self):
        container = tk.Frame(self.root, bg="#282a36") 
        container.pack(expand=True)
        tk.Label(container, text="Select one of your favorite PAU locations:").pack(pady=10)

        btn_frame = tk.Frame(container)
        btn_frame.pack()

        for name, path in self.video_paths.items():
            tk.Button(btn_frame, text=name, command=lambda p=path: self.select_video(p)).pack(side=tk.LEFT, padx=5)

        self.canvas = tk.Canvas(container, width=640, height=480, bg="grey")
        self.canvas.pack(pady=20)

        control_frame = tk.Frame(container)
        control_frame.pack()

        tk.Button(control_frame, text="Detect Objects", command=self.start_detection).pack(side=tk.LEFT, padx=5)
        self.pause_button = tk.Button(control_frame, text="Pause", command=self.toggle_pause)
        self.pause_button.pack(side=tk.LEFT, padx=5)
        tk.Button(control_frame, text="Stop", command=self.stop_video).pack(side=tk.LEFT, padx=5)
        tk.Button(control_frame, text="Close", command=self.close_app).pack(side=tk.LEFT, padx=5)

    def select_video(self, path):
        if os.path.exists(path):
            self.selected_video = path
            messagebox.showinfo("Video Selected", f"Selected video: {os.path.basename(path)}")
        else:
            messagebox.showerror("Error", f"Video file not found: {path}")

    def start_detection(self):
        if not self.selected_video:
            messagebox.showerror("Error", "Please select a video first.")
            return
        
        if self.cap is not None:
            self.cap.release()
            
        self.running = True
        self.paused = False
        self.cap = cv2.VideoCapture(self.selected_video)
        threading.Thread(target=self.process_video, daemon=True).start()

    def toggle_pause(self):
        if self.running:
            self.paused = not self.paused
            self.pause_button.config(text="Continue" if self.paused else "Pause")

    def stop_video(self):
        self.running = False
        self.paused = False
        if self.cap:
            self.cap.release()
            self.cap = None
        self.canvas.delete("all")
        self.current_frame = None
        self.pause_button.config(text="Pause")

    def close_app(self):
        self.stop_video()
        self.root.destroy()

    def process_video(self):
        while self.running and self.cap.isOpened():
            if not self.paused:
                ret, frame = self.cap.read()
                if not ret:
                    break

                frame = cv2.resize(frame, (640, 480))
                frame = detect_objects_yolo(frame)

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=img_pil)

                self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
                self.canvas.image = imgtk
                self.current_frame = frame

            # Add a small delay to prevent high CPU usage
            cv2.waitKey(30)

        if self.cap:
            self.cap.release()
            self.cap = None

if __name__ == "__main__":
    root = tk.Tk()
    app = ObjectDetectionApp(root)
    root.mainloop()